# Files!

This week we're learning the basics of handling files for reading in and writing out.  We're used to our operating systems handling all these things for us, so we need to reproduce those backend mechanisms ourselves.

The basic pattern is:

1. Open the file in whichever mode you need
2. Do stuff to that file.
3. Close the file.

There are ways of handling files in sort of a one shot, but we'll get to that later.

## The cursor

Don't curse the cursor!  When you open a file for reading, there's an invisible cursor that the system places within that file.  It starts at the very beginning of the file and moves forward at your direction.  This means you can direct it to go forward character by character or line by line.  But you can never move backwards!  This means that you cannot 'reread' a file once you've already done a read action to it.  Once the cursor has found the end of the file, it stops and waits.

For example:

1. You open a file for reading
2. You use `.read()` on that file.  The cursor is now at the end of the file.
3. You cannot then use `.readlines()` on it after because `.readlines()` is going to attempt to start reading from the end of that file.  You'll get nothing.

In [1]:
my_file = open('smalltext.txt', 'rt')

all_the_text = my_file.read()

print(all_the_text) # just the first 10 lines, please

maybemore = my_file.read()

print(maybemore)

Hello, I would like to science.
Please show me where the science is.
I am a meat popcicle.



Printing these out sort of hide what's happening in these strings. So let's use some handy interactive stuff.

In [2]:
all_the_text

'Hello, I would like to science.\nPlease show me where the science is.\nI am a meat popcicle.'

In [3]:
maybemore

''

So `maybemore` worked, but is empty.  The cursor had no more text to go through, so it just gave us an empty string.  But let's look at this a different way and see if we can halt the cursor in the middle of a file.

In [4]:
my_file = open('boomboom.txt', 'rt')

for justdothis5times in range(5):
    print(my_file.readline())
    
therest = my_file.read()

A told B, and B told C, "I'll meet you at the top of the coconut tree."

"Wheee!" said D to E F G, "I'll beat you to the top of the coconut tree."

Chicka chicka boom boom! Will there be enough room? Here comes H up the coconut tree,

and I and J and tag-along K, all on their way up the coconut tree.

Chicka chicka boom boom! Will there be enough room? Look who's coming! L M N O P!



In [5]:
print(therest)

And Q R S! And T U V! Still more - W! And X Y Z!
The whole alphabet up the - Oh, no! Chicka chicka... BOOM! BOOM!
Skit skat skoodle doot. Flip flop flee. Everybody running to the coconut tree.
Mamas and papas and uncles and aunts hug their little dears, then dust their pants.
"Help us up," cried A B C.
Next from the pileup skinned-knee D and stubbed-toe E and patched-up F. Then comes G all out of breath.
H is tangled up with I. J and K are about to cry. L is knotted like a tie.
M is looped. N is stopped. O is twisted alley-oop. Skit skat skoodle doot. Flip flop flee.
Look who's coming! It's black-eyed P, Q R S, and loose-tooth T. Then U V W wiggle-jiggle free.
Last to come X Y Z. And the sun goes down on the coconut tree...
But - chicka chicka boom boom! Look, there's a full moon.
A is out of bed, and this is what he said, "Dare double dare, you can't catch me.
Chicka chicka BOOM! BOOM!Chicka chicka BOOM! BOOM!
I'll beat you to the top of the coconut tree."
Chicka chicka BOOM! BOOM!


What happened here?  We can see that the `.readline()` bit grabbed the first 5 lines and then the `.read()` got the rest of the lines.  There also seems to be extra newlines happening in the first section?

I used a for loop with `range(5)` to repeat `.readline()` 5 times.  This meant it acted 5 independent times, so it printed out 5 lines that each ended in `\n`, which means that gets rendered as an extra newline, because there's no text right after it.

At this point, the cursor is sitting at the beginning of line 6 just waiting.  When I call `.read()` it goes through the remaining portion of the file.  There's no extra newlines happening because there's text to sit in those slots.

# A problem to work through

Here's something silly, but it's a task.

Read through a text file and change it so that the lines alternate between upper and lower case (starting with upper), then write out the new file. 

So we want something like this:
```
LINE 1
line 2
LINE 3
line 4
```

Let's break this down:

1. Read in the file
2. Transform the text
3. Write out the file

We can go ahead and set up items 1 and 3.

In [6]:
file_in = open('boomboom.txt', 'rt')

write_me = file_in.read()

file_out = open('newboom.txt', 'wt')

file_out.write(write_me)

file_in.close()
file_out.close()

In the code in the cell just above, we can see that we aren't transforming the text yet, but we are at leat set up for the read in and write out.  While we're playing with things, let's comment out the write stuff and just use print statements.

In [7]:
file_in = open('boomboom.txt', 'rt')

write_me = file_in.read()


print(write_me)
# file_out = open('newboom.txt', 'wt')

# file_out.write(write_me)

file_in.close()
# file_out.close()

A told B, and B told C, "I'll meet you at the top of the coconut tree."
"Wheee!" said D to E F G, "I'll beat you to the top of the coconut tree."
Chicka chicka boom boom! Will there be enough room? Here comes H up the coconut tree,
and I and J and tag-along K, all on their way up the coconut tree.
Chicka chicka boom boom! Will there be enough room? Look who's coming! L M N O P!
And Q R S! And T U V! Still more - W! And X Y Z!
The whole alphabet up the - Oh, no! Chicka chicka... BOOM! BOOM!
Skit skat skoodle doot. Flip flop flee. Everybody running to the coconut tree.
Mamas and papas and uncles and aunts hug their little dears, then dust their pants.
"Help us up," cried A B C.
Next from the pileup skinned-knee D and stubbed-toe E and patched-up F. Then comes G all out of breath.
H is tangled up with I. J and K are about to cry. L is knotted like a tie.
M is looped. N is stopped. O is twisted alley-oop. Skit skat skoodle doot. Flip flop flee.
Look who's coming! It's black-eyed P, Q R S, 

Great! Now let's think of an approach for how to do this.  We know that we can use `.upper()` and `.lower()` to transform the text, but the issue is: how do we alternate lines of text?

Think of it this way: we need to move through the file in terms of pairs.  One upper, one lower then one upper and one lower.

We can use the cursor to our advantage here.  If we're already iterating through reading a file using a for loop, e.g.

```python
for line in file_in:
    print(line)

```

We're just using this for loop as a convenient way to move the cursor one by one.  So can we move thorugh two lines within that for loop?  We can't just use line twice becuase it'll always be the second line, but we have another method in our pockets:  `.readline()`.  As we saw with our `range(5)` example, we can call `.readline()` an arbitrary number of times.  It won't mess up our for loop, but it will give us the next line, and push the cursor forward by one.

In other words:


```python

# let's say the cursor starts at 0

for line in file_in: # cursor + 1; cursor is now 1
    print(line)      # cursor doesn't change; cursor remains 1
    print(file_in.readline()) # cursor + 1; cursor is now 2
    
# now this will run again

# ...

# cursor is at 2 from the previous loop 

for line in file_in: # cursor + 1; cursor is now 3
    print(line)      # cursor doesn't change; cursor remains 3
    print(file_in.readline()) # cursor + 1; cursor is now 4
    
# and so on
```

Let's play with this using just a regular counter and an accumulator pattern.  

In [8]:
total = 0

lotsofones = [1] * 10
print(lotsofones)

for one in lotsofones:
    total = total + one
    total = total + 1
    print(total)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2
4
6
8
10
12
14
16
18
20


So plunking all of this in together:

In [10]:
file_in = open('boomboom.txt', 'rt')

new_text = []

for line in file_in:
    new_text.append(line.upper())
    new_text.append(file_in.readline().lower())

write_me = "".join(new_text) # I retained the newlines as they came in from the original file
                             # so I don't need to put them back in

print(write_me)
# file_out = open('newboom.txt', 'wt')

# file_out.write(write_me)

# file_in.close()
# file_out.close()

A TOLD B, AND B TOLD C, "I'LL MEET YOU AT THE TOP OF THE COCONUT TREE."
"wheee!" said d to e f g, "i'll beat you to the top of the coconut tree."
CHICKA CHICKA BOOM BOOM! WILL THERE BE ENOUGH ROOM? HERE COMES H UP THE COCONUT TREE,
and i and j and tag-along k, all on their way up the coconut tree.
CHICKA CHICKA BOOM BOOM! WILL THERE BE ENOUGH ROOM? LOOK WHO'S COMING! L M N O P!
and q r s! and t u v! still more - w! and x y z!
THE WHOLE ALPHABET UP THE - OH, NO! CHICKA CHICKA... BOOM! BOOM!
skit skat skoodle doot. flip flop flee. everybody running to the coconut tree.
MAMAS AND PAPAS AND UNCLES AND AUNTS HUG THEIR LITTLE DEARS, THEN DUST THEIR PANTS.
"help us up," cried a b c.
NEXT FROM THE PILEUP SKINNED-KNEE D AND STUBBED-TOE E AND PATCHED-UP F. THEN COMES G ALL OUT OF BREATH.
h is tangled up with i. j and k are about to cry. l is knotted like a tie.
M IS LOOPED. N IS STOPPED. O IS TWISTED ALLEY-OOP. SKIT SKAT SKOODLE DOOT. FLIP FLOP FLEE.
look who's coming! it's black-eyed p, q r s, 

'A TOLD B, AND B TOLD C, "I\'LL MEET YOU AT THE TOP OF THE COCONUT TREE."\n"wheee!" said d to e f g, "i\'ll beat you to the top of the coconut tree."\nCHICKA CHICKA BOOM BOOM! WILL THERE BE ENOUGH ROOM? HERE COMES H UP THE COCONUT TREE,\nand i and j and tag-along k, all on their way up the coconut tree.\nCHICKA CHICKA BOOM BOOM! WILL THERE BE ENOUGH ROOM? LOOK WHO\'S COMING! L M N O P!\nand q r s! and t u v! still more - w! and x y z!\nTHE WHOLE ALPHABET UP THE - OH, NO! CHICKA CHICKA... BOOM! BOOM!\nskit skat skoodle doot. flip flop flee. everybody running to the coconut tree.\nMAMAS AND PAPAS AND UNCLES AND AUNTS HUG THEIR LITTLE DEARS, THEN DUST THEIR PANTS.\n"help us up," cried a b c.\nNEXT FROM THE PILEUP SKINNED-KNEE D AND STUBBED-TOE E AND PATCHED-UP F. THEN COMES G ALL OUT OF BREATH.\nh is tangled up with i. j and k are about to cry. l is knotted like a tie.\nM IS LOOPED. N IS STOPPED. O IS TWISTED ALLEY-OOP. SKIT SKAT SKOODLE DOOT. FLIP FLOP FLEE.\nlook who\'s coming! it\'s bl